In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold

In [ ]:
project_dir = "...path_to_project_dir"

In [ ]:
emogi_pred = pd.read_csv(project_dir + 'input_data/cancer_eval/emogi_predictions2.tsv', sep = '\t')

In [ ]:
# read emogi train test sets
train_test = pd.read_csv(project_dir + 'input_data/cancer_eval/emogi_train_test.tsv', sep = '\t')
train_test

In [ ]:
emogi_pred

In [ ]:
emogi_pred.gene_id.unique().shape

In [ ]:
train_test = train_test.merge(emogi_pred, left_on = 'gene_id', right_on = 'gene_id', how = 'left').drop_duplicates()

In [ ]:
folder = project_dir + 'embedding/combination/'
emb = pd.read_csv(folder + 'dtf_gtex_depMap_portT5_lunar-snowflake-239_3787637_embedding.tsv',
                   sep = '\t').set_index('gene_id')

In [ ]:
emb = pd.read_csv(folder + 'combined_STRING.tsv',
                   sep = '\t').set_index('gene_id')

In [ ]:
emb = pd.read_csv(folder + 'combined_STRING_EXP.tsv',
                   sep = '\t').set_index('gene_id')

In [ ]:
df = train_test.merge(emb, on = 'gene_id')

In [ ]:
df.query("dataset == 'train'")

In [ ]:
#kf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 1234)

pred = []

for method in ["score_only", "and_emb", "emb_only"]:


        ## copy p val only in case features is pval
        if method == 'score_only':
            score = df.query("dataset == 'test'")['pred']

        ## otherwise train gradient boosting tree.
        else:
            #boost = HistGradientBoostingClassifier(learning_rate = 0.014 , max_iter = max_iter, max_depth = 4)

            boost = HistGradientBoostingClassifier()

            features_to_drop = ["gene_id", "mask", "y", "dataset", "label"]
            if method == 'emb_only':
                features_to_drop = features_to_drop + ["pred"]

            boost.fit(df.query("dataset == 'train'").drop(features_to_drop, axis=1), df.query("dataset == 'train'").y)
            score = boost.predict_proba(df.query("dataset == 'test'").drop(features_to_drop, axis=1))[:,1]

        pred.append(
            pd.DataFrame({
                "gene_id": df.query("dataset == 'test'")["gene_id"],
                "pred": score,
                "label": df.query("dataset == 'test'")["label"],
                "original_score": df.query("dataset == 'test'")["pred"],
                "method": method, 
            })
        )


pred = pd.concat(pred)
# pred.to_csv(snakemake.output['pred'], sep = '\t', index = False)

In [ ]:
pred

In [ ]:
pred.to_csv(project_dir + 'processed_data/emogi_pred/emogi_test_set.tsv', sep = '\t')

In [ ]:
pred.to_csv(project_dir + 'processed_data/emogi_pred/emogi_STRING_test_set.tsv', sep = '\t')

In [ ]:
pred.to_csv(project_dir + 'processed_data/emogi_pred/emogi_STRING_EXP_test_set.tsv', sep = '\t')